# sts eval

In [1]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import json
import logging
import os
import random
import sys

import numpy as np
import torch
import torch.nn.functional as F

import torch.multiprocessing as mp
import torch.utils.data.distributed

from transformers import (WEIGHTS_NAME, AdamW, BertConfig,
                          BertForTokenClassification, BertTokenizer,
                          get_linear_schedule_with_warmup, 
                          BertPreTrainedModel, BertModel)

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from torch import nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from tqdm import tqdm, trange

from seqeval.metrics import classification_report
from seqeval.metrics import sequence_labeling

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)
import codecs

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
    
import glob
import time

import easydict

import six
import tqdm
from tqdm import tqdm, trange
import collections

from seqeval.metrics import classification_report

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


import re

import shutil

import math
from datetime import datetime
import os
import gzip
import csv

/home/kkm/anaconda3/envs/multibert/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## data class

In [ ]:
class TrainingInstance_ext(object):
    """A single training instance (sentence pair)."""
    def __init__(self, input_ids_0, input_mask_0, segment_ids_0, 
                 input_ids_1, input_mask_1, segment_ids_1, label_ids):
        
        self.input_ids_0 = input_ids_0
        self.input_mask_0 = input_mask_0
        self.segment_ids_0 = segment_ids_0
        
        self.input_ids_1 = input_ids_1
        self.input_mask_1 = input_mask_1
        self.segment_ids_1 = segment_ids_1
        
        self.label_ids = label_ids
        

## BERT with cos distance class

In [ ]:
class SBERT(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        
        #word_embedding_model = models.Transformer(model_name)
        self.bert = BertModel(config, add_pooling_layer=False)
        
        self.ffnn_qury_0 = nn.Linear(config.hidden_size, config.hidden_size//2)
        self.ffnn_qury_1 = nn.Linear(config.hidden_size//2, config.hidden_size)
        
        self.ffnn_cand_0 = nn.Linear(config.hidden_size, config.hidden_size//2)
        self.ffnn_cand_1 = nn.Linear(config.hidden_size//2, config.hidden_size)
        
        self.act1 = nn.ReLU()
        self.act2 = nn.ReLU()
        
        self.init_weights()
        
        self.loss = nn.MSELoss()
        
    def forward(self, sentence_features, label_ids):
        s = 0
        outputs = self.bert(
            input_ids = sentence_features[s]['input_ids'],
            attention_mask=sentence_features[s]['attention_mask'],
            token_type_ids=sentence_features[s]['token_type_ids'],
            position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=None, output_hidden_states=None,return_dict=None,
        )
        sequence_output = outputs[0]
        
        # batchsize x (1), dimension
        cls_embeds_q = torch.zeros(sequence_output.size()[0], sequence_output.size()[2])
        cls_embeds_q = cls_embeds_q.to(self.device)
        for bat in range(sequence_output.size()[0]):
            cls_embeds_q[bat] = sequence_output[bat][0]
        
        # ffnn for query
        m2d_h1_q = self.ffnn_qury_0(cls_embeds_q)
        m2d_h1_q = self.act1(m2d_h1_q) # relu (remove negatives)
        m2d_h2_q = self.ffnn_qury_1(m2d_h1_q)
        m2d_h2_q = self.act1(m2d_h2_q) # relu (remove negatives)
        m2d_h2_q = m2d_h2_q / m2d_h2_q.norm(dim=-1)[:, None].to(self.device) # norm (for cosine sim)
        
        
        # loop 1: sentence_features cands (as much batsize)
        s = 1
        outputs = self.bert(
            input_ids = sentence_features[s]['input_ids'], 
            attention_mask=sentence_features[s]['attention_mask'],
            token_type_ids=sentence_features[s]['token_type_ids'],
            position_ids=None, head_mask=None, inputs_embeds=None, output_attentions=None, output_hidden_states=None,return_dict=None,
        )
        sequence_output = outputs[0]
        # batchsize x (1), dimension
        cls_embeds_c = torch.zeros(sequence_output.size()[0], sequence_output.size()[2])
        cls_embeds_c = cls_embeds_c.to(self.device)
        for bat in range(sequence_output.size()[0]):
            cls_embeds_c[bat] = sequence_output[bat][0]
        
        # ffnn for candidate
        m2d_h1_c = self.ffnn_cand_0(cls_embeds_c)
        m2d_h1_c = self.act1(m2d_h1_c) # relu (remove negatives)
        m2d_h2_c = self.ffnn_cand_1(m2d_h1_c)
        m2d_h2_c = self.act1(m2d_h2_c) # relu (remove negatives)
        m2d_h2_c = m2d_h2_c / m2d_h2_c.norm(dim=-1)[:, None].to(self.device) # norm (for cosine sim)
        cos_sim_fn = nn.CosineSimilarity(dim=1, eps=1e-6)
        cos_distance = 1-cos_sim_fn(m2d_h2_q, m2d_h2_c)
        cos_distance = cos_distance.to(self.device)
        labels = torch.zeros(sequence_output.size()[0])
        labels = labels.to(self.device)
        for bat in range(len(label_ids)):
            labels[bat] = label_ids[bat][0]
        loss = self.loss(labels, cos_distance)
        return loss, cos_distance

## Evaluate

In [ ]:
def evaluate(args):
    ############################################################################
    ##              Multi-GPUs, Distributed settings
    ############################################################################ 
    if args.server_ip and args.server_port:
    # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd
        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()
    
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16))
    
    print("n_gpu: ", n_gpu)
    
    
    ############################################################################
    ##              batch size, gradient_accumulation_steps
    ############################################################################ 
    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            args.gradient_accumulation_steps))
    args.eval_batch_size = args.eval_batch_size // args.gradient_accumulation_steps
    print("args.eval_batch_size: ", args.eval_batch_size)
    
    ###############################################################
    #                Functions for Dataload (load cache data)
    ###############################################################
    def get_file_arrays(data_path):
        data_path = data_path.split(",")
        print("data_path: ", data_path)
        
        cache_files_all = []
        for i in range(len(data_path)):
            path = data_path[i].strip()
#             print("data_path[i]: ", path)
            files = glob.glob(path)
#             print("len(files): ", len(files))
            cache_files_all = cache_files_all + files
        cache_files_all.sort()
        
        print("Got " + str(len(cache_files_all)) + " cache_files")
        
        return cache_files_all
    
    def read_files(cache_files_pieces):
        eval_features = []
        for c in range(len(cache_files_pieces)):
            with open(cache_files_pieces[c], 'rb') as input:
                eval_features = eval_features + pickle.load(input)
            print("cache_files_pieces[c]: ", cache_files_pieces[c])
        print("len(eval_features): ", len(eval_features))        
        
        num_eval_examples = len(eval_features)
        
        all_input_ids_0 = torch.tensor([f.input_ids_0 for f in eval_features], dtype=torch.long)
        all_input_mask_0 = torch.tensor([f.input_mask_0 for f in eval_features], dtype=torch.long)
        all_segment_ids_0 = torch.tensor([f.segment_ids_0 for f in eval_features], dtype=torch.long)
        
        all_input_ids_1 = torch.tensor([f.input_ids_1 for f in eval_features], dtype=torch.long)
        all_input_mask_1 = torch.tensor([f.input_mask_1 for f in eval_features], dtype=torch.long)
        all_segment_ids_1 = torch.tensor([f.segment_ids_1 for f in eval_features], dtype=torch.long)
        
        all_label_ids = torch.tensor([f.label_ids for f in eval_features], dtype=torch.long)
        
        # squeeze
        if all_input_ids_0.size()[0]>1:
            all_input_ids_0 = torch.squeeze(all_input_ids_0)
            all_input_mask_0 = torch.squeeze(all_input_mask_0)
            all_segment_ids_0 = torch.squeeze(all_segment_ids_0)
        
        if all_input_ids_1.size()[0]>1:
            all_input_ids_1 = torch.squeeze(all_input_ids_1)
            all_input_mask_1 = torch.squeeze(all_input_mask_1)
            all_segment_ids_1 = torch.squeeze(all_segment_ids_1)
        
        if all_label_ids.size()[0]>1:
            all_label_ids = torch.squeeze(all_label_ids)
        
        eval_data = TensorDataset(
            all_input_ids_0, all_input_mask_0, all_segment_ids_0, 
            all_input_ids_1, all_input_mask_1, all_segment_ids_1, 
            all_label_ids            
        )

        if args.local_rank == -1:
            eval_sampler = SequentialSampler(eval_data)
        else:
            eval_sampler = DistributedSampler(eval_data)
        
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
        
        return eval_dataloader, num_eval_examples
    
    def read_file(cache_file):
        eval_features = []
        with open(cache_file, 'rb') as input:
            eval_features = eval_features + pickle.load(input)

        num_eval_examples = len(eval_features)
        
        all_input_ids_0 = torch.tensor([f.input_ids_0 for f in eval_features], dtype=torch.long)
        all_input_mask_0 = torch.tensor([f.input_mask_0 for f in eval_features], dtype=torch.long)
        all_segment_ids_0 = torch.tensor([f.segment_ids_0 for f in eval_features], dtype=torch.long)
        
        all_input_ids_1 = torch.tensor([f.input_ids_1 for f in eval_features], dtype=torch.long)
        all_input_mask_1 = torch.tensor([f.input_mask_1 for f in eval_features], dtype=torch.long)
        all_segment_ids_1 = torch.tensor([f.segment_ids_1 for f in eval_features], dtype=torch.long)
        
        all_label_ids = torch.tensor([f.label_ids for f in eval_features], dtype=torch.long)
        
        # squeeze
        if all_input_ids_0.size()[0]>1:
            all_input_ids_0 = torch.squeeze(all_input_ids_0)
            all_input_mask_0 = torch.squeeze(all_input_mask_0)
            all_segment_ids_0 = torch.squeeze(all_segment_ids_0)
        
        if all_input_ids_1.size()[0]>1:
            all_input_ids_1 = torch.squeeze(all_input_ids_1)
            all_input_mask_1 = torch.squeeze(all_input_mask_1)
            all_segment_ids_1 = torch.squeeze(all_segment_ids_1)
        
        if all_label_ids.size()[0]>1:
            all_label_ids = torch.squeeze(all_label_ids)
            
        eval_data = TensorDataset(
            all_input_ids_0, all_input_mask_0, all_segment_ids_0, 
            all_input_ids_1, all_input_mask_1, all_segment_ids_1, 
            all_label_ids            
        )
        
        if args.local_rank == -1:
            eval_sampler = SequentialSampler(eval_data)
        else:
            eval_sampler = DistributedSampler(eval_data)
        
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size)
        
        return eval_dataloader, num_eval_examples
    
    ############################################################################
    ##                    Load Training data
    ############################################################################
    label_list = ["0", "1"]
    num_labels = len(label_list)
    
    
    ###############################################################
    #            Dataload (load cache data) - Train data
    ###############################################################
    cache_files = get_file_arrays(args.data_dir)
    print("cache_files: ", cache_files)
    

    ############################################################################
    ##                      Prepare model
    ############################################################################
    print("loading weights from checkpoint (", args.checkpoint, ")")
    config = BertConfig.from_pretrained(args.checkpoint, num_labels=num_labels)
    model = SBERT.from_pretrained(args.checkpoint,
              from_tf = False,
              config = config)
    print("loaded weights from checkpoint (", args.checkpoint, ")")
    
    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    print("device: ", device)
    
    model.to(device)
    
    ############################################################################
    ##              save_model
    ############################################################################
    def save_model(args, global_step):
        if not os.path.exists(args.output_dir+"/"+str(global_step)):
            os.makedirs(args.output_dir+"/"+str(global_step))

        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        model_to_save.save_pretrained(args.output_dir+"/"+str(global_step))

    def make_sentence_feature(input_ids, input_mask, segment_ids):
        sentencefeature = {}
        sentencefeature['input_ids'] = input_ids
        sentencefeature['attention_mask'] = input_mask
        sentencefeature['token_type_ids'] = segment_ids
        return sentencefeature
    
    ############################################################################
    ##              Start training
    ############################################################################
    global_step = -1
    nb_tr_steps = 0
    tr_loss = 0
    
    args.save_file_limit
    min_train_loss = 1000000
    save_checkpoints = []
    save_loss = []
    
    model.eval()
    
    for c in range(len(cache_files)):
        filename = cache_files[c].split("/")[-1].split(".")[0]
        eval_dataloader, num_eval_examples = read_file(cache_files[c])
        outtext = []
        
        for step, batch in enumerate(tqdm(eval_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids_0, input_mask_0, segment_ids_0, \
            input_ids_1, input_mask_1, segment_ids_1, label_ids = batch
            
            sentence_features = []
            sentence_features.append(make_sentence_feature(input_ids_0, input_mask_0, segment_ids_0))
            sentence_features.append(make_sentence_feature(input_ids_1, input_mask_1, segment_ids_1))            
            
            loss, props = model(sentence_features, label_ids)
            
            input_ids_0 = input_ids_0.detach().cpu()
            input_ids_1 = input_ids_1.detach().cpu()
            label_ids   = label_ids.detach().cpu()
            loss        = loss.detach().cpu()
            props       = props.detach().cpu()
            
            assert len(input_ids_0)==len(input_ids_1)
            for b in range(len(input_ids_0)):
                cosim = str(props[b].item())
                label = str(label_ids[b][0].item()) # 0-th: gold standard
                text0 = ",".join([str(idx) for idx in input_ids_0[b].tolist()])
                text1 = ",".join([str(idx) for idx in input_ids_1[b].tolist()])
                outtext.append(cosim+"\t"+label+"\t"+text0+"\t"+text1)
            
            
        test_mode = args.data_dir.split("/")[-2]
        if not os.path.exists(args.output_dir+"/"+str(test_mode)+"_pred"):
            os.makedirs(args.output_dir+"/"+str(test_mode)+"_pred")

        file = open(args.output_dir+"/"+str(test_mode)+"_pred"+"/"+str(filename)+".txt", "w")
        file.write("\n".join(outtext))
        file.close()
    

In [ ]:
def main(args):
    parser = argparse.ArgumentParser()
    
    print("args.output_dir: ", args.output_dir)
    
    # check directories
    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")
    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
            
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    
    if args.do_train:
        train(args)
    
    if args.do_eval:
        evaluate(args)

# finetune

In [ ]:
data_dirs = [
    './cache/bertbase_cased/test/*.cache',
    './cache/mbert_cased/test/*.cache',
    './cache/biobert/test/*.cache',
    './cache/kobert/test/*.cache',
]

checkpoints = [
    "./finetuned/ver9.1.4_521121_epoch2/4597",
    "./finetuned/ver8.1.4_1142642_epoch2/4597",
    './finetuned/ver11.1.4_521079_epoch2/4597',
    './finetuned/ver12.1.4_407013_epoch2/4597',
]


assert len(data_dirs)==len(checkpoints)

for d in range(len(data_dirs)):
    otherbert = data_dirs[d].split("/")[2]
    output_dir = "./data/08_preds/"+checkpoints[d].split("/")[-2]
                                   
    print("output_dir: ", output_dir)
                                   
    args = easydict.EasyDict({
        'data_dir':data_dirs[d],
        'eval_batch_size':4, 
        'gradient_accumulation_steps':1,
        'save_globalstep':1,
        'checkpoint':checkpoints[d],
        'output_dir':output_dir,
        'save_step':1,
        'num_files_on_memory':1,
        
        'do_train':False,
        'do_eval':True,
        'do_lower_case':True,
        'fp16_allreduce':False,
        'seed':42,
        'no_cuda':False,
        'use_adasum':False,
        'learning_rate':3e-05,
        'warmup_proportion':1.00,
        'num_train_epochs':2,
        'save_file_limit':1,
        
        'weight_decay':0.01,
        'adam_epsilon':1e-8,
        'gradient_predivide_factor':1.0,
        'fp16':False,
        'max_grad_norm':1.0,
        'local_rank':-1,
        'fp16_opt_level':'O1',
        'loss_scale':0,
        
        'server_ip':None,
        'server_port':None,
        'cuda':True,
    })
    
    
    # configs
    if "multilingualbert" == str(data_dirs[d].split("/")[2]):
        print("multilingualbert configs")
        # multilingual bert config
        args["attention_probs_dropout_prob"] = 0.1
        args["directionality"] = "bidi"
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["max_position_embeddings"] = 512
        args["num_attention_heads"] =  12
        args["num_hidden_layers"] = 12
        args["pooler_fc_size"] = 768
        args["pooler_num_attention_heads"] = 12
        args["pooler_num_fc_layers"] = 3
        args["pooler_size_per_head"] = 128
        args["pooler_type"] = "first_token_transform"
        args["type_vocab_size"] = 2
        args["vocab_size"] = 119547
        args["cls_id"] = 101
        args["sep_id"] = 102
    # configs
    elif "biobert" == str(data_dirs[d].split("/")[2]):
        # BioBERT
        print("biobert configs")
        args["attention_probs_dropout_prob"]= 0.1
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["max_position_embeddings"] = 512
        args["num_attention_heads"] = 12
        args["num_hidden_layers"] = 12
        args["type_vocab_size"] = 2
        args["vocab_size"] = 28996
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "kobert" == str(data_dirs[d].split("/")[2]):
        print("kobert configs")
        args["attention_probs_dropout_prob"]= 0.1
        args["gradient_checkpointing"]= False
        args["hidden_act"]= "gelu"
        args["hidden_dropout_prob"]= 0.1
        args["hidden_size"]= 768
        args["initializer_range"]= 0.02
        args["intermediate_size"]= 3072
        args["layer_norm_eps"]= 1e-12
        args["max_position_embeddings"]= 512
        args["model_type"]= "bert"
        args["num_attention_heads"]= 12
        args["num_hidden_layers"]= 12
        args["pad_token_id"]= 1
        args["type_vocab_size"]= 2
        args["vocab_size"]= 8002
        args["author"]= "Heewon Jeon(madjakarta@gmail.com)"
        args["kobert_version"]= 1.0
        args["cls_id"] = 2
        args["sep_id"] = 3
        
    elif "bertbase" == str(data_dirs[d].split("/")[2]):
        args['vocab_size']=30522 # bert-base-uncased
        args['hidden_size']=768
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['hidden_act']='gelu'
        args['intermediate_size']=3072
        args['hidden_dropout_prob']=0.1
        args['attention_probs_dropout_prob']=0.1
        args['max_position_embeddings']=512
        args['type_vocab_size']=2
        args['initializer_range']=0.02
        args['layer_norm_eps']=1e-12
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    
    main(args)